In [1]:
import torch
import numpy as np
import argparse
import os.path
import math
import sys

from tqdm import tqdm
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_undirected, from_scipy_sparse_matrix, dense_to_sparse, is_undirected, coalesce
from torch_geometric.utils import contains_isolated_nodes, degree, remove_self_loops, k_hop_subgraph

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cpu


In [3]:
def set_random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

### Parameter Setting

In [4]:
parser = argparse.ArgumentParser(description='Hyperlink Prediction by Extracting Subgraph Structural Features')
parser.add_argument('--data', type=str,  default='email-Enron', help='graph name')
parser.add_argument('--data_split',type=str, default='0', help='5 splits')
#training/validation/test divison and ratio
parser.add_argument('--val_ratio', type=float, default=0.2,
                    help='ratio of the validation set')
#Model and Training
parser.add_argument('--seed', type=int, default=1,
                    help='random seed (default: 1)')
parser.add_argument('--lr', type=float, default=1e-4,
                    help='learning rate')
parser.add_argument('--weight_decay', type=float, default=5e-4)
parser.add_argument('--walk_len', type=int, default=6, help='cutoff in the length of walks')
parser.add_argument('--num_hops', type=int, default=2)
parser.add_argument('--hidden_channels', type=int, default=32)
parser.add_argument('--batch_size', type=int, default = 32)
parser.add_argument('--epoch_num', type=int, default= 1500)

parser.add_argument('--log', type=str, default=None,
                    help='log by tensorboard, default is None')

args = parser.parse_args(args=[])

print(args)

Namespace(data='email-Enron', data_split='0', val_ratio=0.2, seed=1, lr=0.0001, weight_decay=0.0005, walk_len=6, num_hops=2, hidden_channels=32, batch_size=32, epoch_num=1500, log=None)


In [5]:
def obtain_a_p(data):
    """
    this function is used to obtain the adjacent matrix of the clique/line expansion graph.
    # we tried to use the line graph to learn the feature of focal hyperedge, but the results are not good.
    # so data. _p are actually not used.
    """
    
    # the adjacency matrix
    index = torch.cat((data.node_idx, data.edge_idx), dim=0)
    value = torch.ones_like(data.edge_idx).view(-1).to(torch.float)

    i = torch.sparse_coo_tensor(index, value, (data.node_num, data.edge_num))
    a = torch.sparse.mm(i, i.t())
    a = a.coalesce()
    data.edge_index_a, data.edge_weight_a = a.indices().to(torch.long), a.values().to(torch.float)
    data.edge_index_a, data.edge_weight_a = remove_self_loops(data.edge_index_a, data.edge_weight_a)

    # the intersection profile, i.e., the line graph's adjacent matrix
    p = torch.sparse.mm(i.t(), i)
    p = p.coalesce()
    data.edge_index_p, data.edge_weight_p = p.indices().to(torch.long), p.values().to(torch.float)
    data.edge_index_p, data.edge_weight_p = remove_self_loops(data.edge_index_p, data.edge_weight_p)
    data.it = i.t()

    return data

In [6]:
def sys_normalized_adjacency(adj):
    adj = sp.coo_matrix(adj)
    row_sum = np.array(adj.sum(1))
    row_sum = (row_sum == 0) * 1 + row_sum
    d_inv_sqrt = np.power(row_sum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [7]:
def obtain_walk_profile(data, args):
    """
    for small graphs, using cpu is faster
    for large graphs, using gpu is faster
    
    by default, we consider two cases in our edge weakening process.
    \alpha_set = {0,1}
    """
    
    walk_profile = list()
    row, col = data.edge_index
    row, col = row.view(-1), col.view(-1)
    if data.edge_index.size(1) == 0:
        return torch.zeros(1, args.walk_len * 10)

    g = sp.csr_matrix((data.edge_weight.view(-1), (row, col)), shape=(data.num_nodes, data.num_nodes))
    mat_p = sys_normalized_adjacency(g)
        
    mat_p = sparse_mx_to_torch_sparse_tensor(mat_p)
    x_p = torch.eye(data.num_nodes, dtype=torch.float32)
    # mat_p = sparse_mx_to_torch_sparse_tensor(mat_p).cuda()
    # x_p = torch.eye(data.num_nodes, dtype=torch.float32).cuda()


    # for adjacency matrix, the edge weight substract by one
    edge_weight = data.edge_weight
    edge_weight[data.edge_mask] -= 1.0

    g1 = sp.csr_matrix((edge_weight.view(-1), (row, col)), shape=(data.num_nodes, data.num_nodes))
    mat_m1 = sys_normalized_adjacency(g1)


    mat_m1 = sparse_mx_to_torch_sparse_tensor(mat_m1)
    x_m1 = torch.eye(data.num_nodes, dtype=torch.float32)
    # mat_m1 = sparse_mx_to_torch_sparse_tensor(mat_m1).cuda()
    # x_m1 = torch.eye(data.num_nodes, dtype=torch.float32).cuda()

    # for adjacency matrix, set edge weights to zero
    edge_weight = data.edge_weight
    edge_weight[data.edge_mask] = 0

    g2 = sp.csr_matrix((edge_weight.view(-1), (row, col)), shape=(data.num_nodes, data.num_nodes))
    mat_m2 = sys_normalized_adjacency(g2)
    mat_m2 = sparse_mx_to_torch_sparse_tensor(mat_m2)
    x_m2 = torch.eye(data.num_nodes, dtype=torch.float32)
    # mat_m2 = sparse_mx_to_torch_sparse_tensor(mat_m2).cuda()
    # x_m2 = torch.eye(data.num_nodes, dtype=torch.float32).cuda()

    for i in range(args.walk_len):
        x_p = torch.spmm(mat_p, x_p)
        x_m1 = torch.spmm(mat_m1, x_m1)
        x_m2 = torch.spmm(mat_m2, x_m2)

        walk_profile.append(torch.diagonal(x_p)[data.node_mask].mean())
        walk_profile.append(x_p[data.node_mask, :][:, data.node_mask].mean())
        walk_profile.append(torch.diagonal(x_m1)[data.node_mask].mean())
        walk_profile.append(x_m1[data.node_mask, :][:, data.node_mask].mean())
        walk_profile.append(torch.diagonal(x_p).mean() - torch.diagonal(x_m1).mean())
        walk_profile.append(torch.diagonal(x_m2)[data.node_mask].mean())
        walk_profile.append(x_m2[data.node_mask, :][:, data.node_mask].mean())
        walk_profile.append(torch.diagonal(x_p).mean() - torch.diagonal(x_m2).mean())

    walk_profile = torch.tensor(walk_profile, dtype=torch.float32).view(1, -1)
    return walk_profile

In [8]:
def ego_graph_minus(data, edge, h, args):
    # the ego-graph in the adjaceny matarix
    num_nodes_a = torch.max(data.edge_index_a) + 1
    sub_nodes, edge_index_a, mapping, edge_mask = k_hop_subgraph(edge.view(-1), args.num_hops, data.edge_index_a,
                                                                 relabel_nodes=True, num_nodes=num_nodes_a)
    num_nodes_a = torch.max(edge_index_a) + 1
    node_mask_a = torch.zeros(num_nodes_a, dtype=torch.bool)

    # mask of the nodes contained in a hyperedge
    node_mask_a[mapping] = True
    edge_weight_a = data.edge_weight_a[edge_mask].view(-1, 1)

    # mask of the edges run among the nodes
    edge_mask_row = torch.zeros(edge_index_a.size(1), dtype=torch.bool)
    edge_mask_col = torch.zeros(edge_index_a.size(1), dtype=torch.bool)

    row, col = edge_index_a
    torch.index_select(node_mask_a, 0, row, out=edge_mask_row)
    torch.index_select(node_mask_a, 0, col, out=edge_mask_col)
    edge_mask_a = torch.logical_and(edge_mask_row, edge_mask_col)

    data_a = Data(edge_index=edge_index_a, edge_attr=edge_weight_a, num_nodes=num_nodes_a)
    data_a.edge_weight = edge_weight_a
    data_a.edge_mask = edge_mask_a
    data_a.node_mask = node_mask_a

    return data_a


def ego_graph_plus(data, edge, args):
    # add a hyperedge in the adjacency matrix
    node_paris = torch.combinations(edge.view(-1)).transpose(0, 1)
    edge_index_a = torch.cat((data.edge_index_a, node_paris), dim=1)
    edge_index_a = torch.cat((edge_index_a, node_paris[[1, 0], :]), dim=1)
    edge_weight_a = torch.cat((data.edge_weight_a, torch.ones(node_paris.size(1) * 2)), dim=0)
    edge_index_a, edge_weight_a = coalesce(edge_index_a, edge_weight_a)

    num_nodes_a = torch.max(edge_index_a) + 1
    sub_nodes, edge_index_a, mapping, edge_mask = k_hop_subgraph(edge.view(-1), args.num_hops, edge_index_a,
                                                                 relabel_nodes=True, num_nodes=num_nodes_a)
    num_nodes_a = torch.max(edge_index_a) + 1
    node_mask_a = torch.zeros(num_nodes_a, dtype=torch.bool)
    node_mask_a[mapping] = True
    edge_mask_row = torch.zeros(edge_index_a.size(1), dtype=torch.bool)
    edge_mask_col = torch.zeros(edge_index_a.size(1), dtype=torch.bool)

    row, col = edge_index_a
    torch.index_select(node_mask_a, 0, row, out=edge_mask_row)
    torch.index_select(node_mask_a, 0, col, out=edge_mask_col)
    edge_mask_a = torch.logical_and(edge_mask_row, edge_mask_col)  # node pairs run inside the hyper edge
    edge_weight_a = edge_weight_a[edge_mask].view(-1, 1)

    data_a = Data(edge_index=edge_index_a, edge_attr=edge_weight_a, num_nodes=num_nodes_a)
    data_a.edge_weight = edge_weight_a
    data_a.edge_mask = edge_mask_a
    data_a.node_mask = node_mask_a

    return data_a

In [9]:
def load_splitted_data(args):
    par_dir = os.path.abspath('')
    data_name = args.data + "_split_" + args.data_split
    data_dir = os.path.join(par_dir, "data/{}.npz".format(data_name))
    data = np.load(data_dir, allow_pickle=True)
    train_data = data['arr_0']
    train_label = data['arr_1']
    test_data = data['arr_2']
    test_label = data['arr_3']

    # Convert to Torch tensors
    train_label = torch.tensor(train_label, dtype=torch.long)
    test_lb = torch.tensor(test_label, dtype=torch.long)

    pos_ind = torch.where(train_label == 1)[0]
    neg_ind = torch.where(train_label == 0)[0]
    pos_val_size = int(pos_ind.size(0) * args.val_ratio)
    neg_val_size = int(neg_ind.size(0) * args.val_ratio)
    is_train = torch.ones_like(train_label).to(torch.bool)
    perm = torch.randperm(pos_ind.size(0))
    is_train[pos_ind[perm[:pos_val_size]]] = False
    perm = torch.randperm(neg_ind.size(0))
    is_train[neg_ind[perm[:neg_val_size]]] = False

    train_edges = list()
    val_edges = list()
    test_edges = list()
    train_lb = list()
    val_lb = list()
    train_pos_id = list()
    val_pos_id = list()

    # contruct the observed hypergraph
    edge_id = -1
    num_nodes = 0
    node_idx, edge_idx = torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long)
    for ind, edge in enumerate(train_data):
        nodes = torch.tensor(edge).view(1, -1).to(torch.long)
        max_node = torch.max(nodes)
        if max_node > num_nodes:
            num_nodes = max_node
        if is_train[ind]:
            train_edges.append(nodes)
            train_lb.append(train_label[ind])
            if train_label[ind] == 1:
                edge_id += 1
                edges = torch.full(nodes.size(), edge_id).to(torch.long)
                node_idx = torch.cat((node_idx, nodes), dim=1)
                edge_idx = torch.cat((edge_idx, edges), dim=1)
                train_pos_id.append(edge_id)
        else:
            val_edges.append(nodes)
            val_lb.append(train_label[ind])
            if train_label[ind] == 1:
                edge_id += 1
                edges = torch.full(nodes.size(), edge_id).to(torch.long)
                node_idx = torch.cat((node_idx, nodes), dim=1)
                edge_idx = torch.cat((edge_idx, edges), dim=1)
                val_pos_id.append(edge_id)

    for ind, edge in enumerate(test_data):
        max_node = torch.max(nodes)
        if max_node > num_nodes:
            num_nodes = max_node
        nodes = torch.tensor(edge).view(1, -1).to(torch.long)
        test_edges.append(nodes)

    data = Data()
    data.node_idx = node_idx
    data.edge_idx = edge_idx
    data.node_num = int(num_nodes + 1)
    data.edge_num = int(edge_id + 1)
    data = obtain_a_p(data)

    train_lb = torch.tensor(train_lb).to(torch.long)
    val_lb = torch.tensor(val_lb).to(torch.long)

    return data, train_edges, val_edges, test_edges, train_lb, val_lb, test_lb, train_pos_id, val_pos_id

In [10]:
### Data Loader and Feature Extraction

In [11]:
data, train_edges, val_edges, test_edges, train_lb, val_lb, test_lb, train_pos_ids, val_pos_ids = load_splitted_data(args)
"""
we strictly divide train/val/test, only train hyperedges are used in the construction of observed hypergraph. 
given an abtrary hyperedge, we extract features based on this hyperedge and the observed hypergraph.

"""

set_random_seed(args.seed)
train_data = torch.tensor([])
val_data = torch.tensor([])
test_data = torch.tensor([])

train_pos_id = -1
train_labels = []
for ind, edge in enumerate(tqdm(train_edges)):
    if edge.shape[1] == 1:
        continue
    else:
        if train_lb[ind] == 1:
            train_labels.append(1)
            train_pos_id = train_pos_ids.pop(0)
            data_a = ego_graph_minus(data, edge, train_pos_id, args)
        else:
            train_labels.append(0)
            data_a = ego_graph_plus(data, edge, args)
        walk_profile = obtain_walk_profile(data_a, args)
        train_data = torch.cat((train_data, walk_profile), dim=0)

val_labels = []
for ind, edge in enumerate(tqdm(val_edges)):
    if edge.shape[1] == 1:
        continue
    else:
        if val_lb[ind] == 1:
            val_labels.append(1)
            val_pos_id = val_pos_ids.pop(0)
            data_a = ego_graph_minus(data, edge, val_pos_id, args)
        else:
            val_labels.append(0)
            data_a = ego_graph_plus(data, edge, args)
        walk_profile = obtain_walk_profile(data_a, args)
        val_data = torch.cat((val_data, walk_profile), dim=0)

test_labels = []
for ind, edge in enumerate(tqdm(test_edges)):
    if edge.shape[1] == 1:
        continue
    else:
        test_labels.append(test_lb[ind])
        data_a = ego_graph_plus(data, edge, args)
        walk_profile = obtain_walk_profile(data_a, args)
        test_data = torch.cat((test_data, walk_profile), dim=0)

train_lb = torch.tensor(train_labels).to(torch.long)
val_lb = torch.tensor(val_labels).to(torch.long)
test_lb = torch.tensor(test_labels).to(torch.long)

print('<<Complete generating training data>>')

print("num_train_edges: ", len(train_edges))
print("num_val_edges: ", len(val_edges))
print("num_test_edges: ", len(test_edges))

print ("-"*42+'Model and Training'+"-"*45)
print ("{:<13}|{:<13}|{:<13}|{:<8}|{:<13}|{:<15}"\
    .format('Learning Rate','Weight Decay','Batch Size','Epoch',\
        'Walk Length','Hidden Channels'))
print ("-"*105)

print ("{:<13}|{:<13}|{:<13}|{:<8}|{:<13}|{:<15}"\
    .format(args.lr,args.weight_decay, str(args.batch_size),\
        args.epoch_num,args.walk_len, args.hidden_channels))
print ("-"*105)

100%|█████████████████████████████████████████| 408/408 [00:04<00:00, 87.52it/s]

<<Complete generating training data>>
num_train_edges:  1988
num_val_edges:  496
num_test_edges:  408
------------------------------------------Model and Training---------------------------------------------
Learning Rate|Weight Decay |Batch Size   |Epoch   |Walk Length  |Hidden Channels
---------------------------------------------------------------------------------------------------------
0.0001       |0.0005       |32           |1500    |6            |32             
---------------------------------------------------------------------------------------------------------


In [12]:
class MLP(torch.nn.Module):
    # adopt a MLP as classifier for graphs
    def __init__(self,input_size, hidden_channels):
        super(MLP, self).__init__()
        self.nn = nn.BatchNorm1d(input_size)
        self.linear1 = torch.nn.Linear(input_size,hidden_channels*20)
        self.linear2 = torch.nn.Linear(hidden_channels*20,hidden_channels*10)
        self.linear3 = torch.nn.Linear(hidden_channels*10,hidden_channels)
        self.linear4 = torch.nn.Linear(hidden_channels,hidden_channels)
        self.linear5 = torch.nn.Linear(hidden_channels,1)
        self.act= nn.ReLU()

    def forward(self, x):
        out= self.nn(x)
        out= self.linear1(out)
        out = self.act(out)
        out= self.linear2(out)
        out = self.act(out)
        out = self.linear3(out)
        out = self.act(out)
        out = self.linear4(out)
        out = self.act(out)
        out = F.dropout(out, p=0.7, training=self.training)
        out = self.linear5(out)
        return out

In [13]:
def train(data, lbs):
    classifier.train()
    torch.cuda.empty_cache()
    out = classifier(data)
    loss = criterion(out.view(-1), lbs.view(-1).to(torch.float))
    optimizer_classifier.zero_grad()
    loss.backward()
    optimizer_classifier.step()
    loss_epoch = loss.item()
    return loss_epoch

def test(data, lbs):
    classifier.eval()
    with torch.no_grad():
        out = classifier(data)
        loss = criterion(out.view(-1), lbs.view(-1).to(torch.float))
        scores = out.cpu().clone().detach()
        tpred = scores.flatten()
        num_predictions = int(sum(lbs))
        cut = np.partition(tpred, -num_predictions)[-num_predictions]
        tpred = torch.Tensor(tpred)
        pred = torch.where(tpred >= cut, torch.ones_like(tpred), torch.zeros_like(tpred)).view(-1)
        # pred = torch.sigmoid(scores)
        # pred = torch.where(pred>0.6, torch.ones_like(pred), torch.zeros_like(pred)).view(-1)
        f1 = f1_score(np.array(lbs.cpu().tolist()), np.array(pred.cpu().tolist()))
        auc = roc_auc_score(np.array(lbs.cpu().tolist()), scores)
        ap = average_precision_score(np.array(lbs.cpu().tolist()), scores)
        return f1, auc, ap, loss.item()

In [14]:
walk_len = args.walk_len
hidden_channels=args.hidden_channels
lr=args.lr
weight_decay=args.weight_decay

torch.cuda.empty_cache()

args.num_features = train_data.shape[1]


torch.cuda.empty_cache()
print("Dimention of features after concatenation: ", args.num_features)
set_random_seed(args.seed)


classifier = MLP(train_data.size(1), args.hidden_channels).to(device)
optimizer_classifier = torch.optim.Adam(classifier.parameters(), lr=lr, weight_decay=weight_decay)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.0]).to(device))

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)
train_lb = train_lb.to(device)
val_lb = val_lb.to(device)
test_lb = test_lb.to(device)

Dimention of features after concatenation:  48


In [15]:
best_from_val = 0
best_val_f1 = 0

"""
we use full-epoch training.
during the test phase, we assume the number of predictions are given.
this assumption is generally used in (hyper-)link prediction studies.
"""


for epoch in range(0, args.epoch_num):
    train_loss = train(train_data, train_lb)
    frac = sum(train_lb)/len(train_lb)
    val_f1, val_auc, val_ap, val_loss = test(val_data, val_lb)
    test_f1, test_auc, test_ap, test_loss = test(test_data, test_lb)
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_from_val = test_f1
        best_auc = test_auc
        best_ap = test_ap
    if epoch % 50 ==0:
        print(f'Epoch: {epoch:03d}, Loss : {train_loss:.4f}, Test AUC: {test_auc:.4f}, Test AP: {test_ap:.4f}, Test f1: {test_f1:.4f}, Picked f1: {best_from_val:.4f}')

Epoch: 000, Loss : 1.1646, Test AUC: 0.6415, Test AP: 0.2511, Test f1: 0.2647, Picked f1: 0.2647
Epoch: 050, Loss : 1.0621, Test AUC: 0.8122, Test AP: 0.5271, Test f1: 0.5735, Picked f1: 0.5735
Epoch: 100, Loss : 0.8917, Test AUC: 0.8269, Test AP: 0.6037, Test f1: 0.6176, Picked f1: 0.6176
Epoch: 150, Loss : 0.8275, Test AUC: 0.8399, Test AP: 0.6317, Test f1: 0.6176, Picked f1: 0.6176
Epoch: 200, Loss : 0.7477, Test AUC: 0.8639, Test AP: 0.6601, Test f1: 0.6324, Picked f1: 0.6176
Epoch: 250, Loss : 0.7177, Test AUC: 0.8823, Test AP: 0.7100, Test f1: 0.6324, Picked f1: 0.6324
Epoch: 300, Loss : 0.6937, Test AUC: 0.8859, Test AP: 0.7224, Test f1: 0.6176, Picked f1: 0.6324
Epoch: 350, Loss : 0.6443, Test AUC: 0.8892, Test AP: 0.7388, Test f1: 0.6471, Picked f1: 0.6176
Epoch: 400, Loss : 0.6296, Test AUC: 0.8917, Test AP: 0.7521, Test f1: 0.6618, Picked f1: 0.6176
Epoch: 450, Loss : 0.5843, Test AUC: 0.8980, Test AP: 0.7691, Test f1: 0.6618, Picked f1: 0.6176
Epoch: 500, Loss : 0.5869, Tes